<a href="https://colab.research.google.com/github/Byte-framework/Hartree-Fock/blob/main/Hartree_Fock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math


#Step 1



In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reading Nuclear Repulsion Energy
fenuc = open("/content/drive/MyDrive/Hartree Fock/enuc.dat","r")


#Step 2
Done

In [ ]:
# Reading Overlap Matrix,Kinetic Energy and Nuclear attraction integrals; forming the core Hamiltonian

fs = open("/content/drive/MyDrive/Hartree Fock/s.dat","r")
fke = open("/content/drive/MyDrive/Hartree Fock/ke.dat","r")
fv = open("/content/drive/MyDrive/Hartree Fock/pe_v.dat","r")

hc = np.zeros((7,7))

for k in fke:
  kl=k.split()
  hc[int(kl[0])-1][int(kl[1])-1] = float(kl[2])
  if int(kl[0])!=int(kl[1]):
    hc[int(kl[1])-1][int(kl[0])-1] = float(kl[2])

for p in fv:
  pl=p.split()
  hc[int(pl[0])-1][int(pl[1])-1] += float(pl[2])
  if int(pl[0])!=int(pl[1]):
    hc[int(pl[1])-1][int(pl[0])-1] += float(pl[2])

#print(hc)










#Step 3
Used one big array & also implemented a dictionary.

Done

In [ ]:
# Reading the two electron integral and storing them in a 1D array and in a dictionary

ftei = open("/content/drive/MyDrive/Hartree Fock/two_integ.dat","r")
tei = np.zeros((406))
ti = dict()
for t in ftei:
  tl=t.split()
  for i in range(4):
    tl[i] = int(tl[i])
  tl[4] = float(tl[4])
  b = int(tl[0]-1)*(int(tl[0]-1) + 1)/2 + int(tl[1]-1)
  c = int(tl[2]-1)*(int(tl[2]-1) + 1)/2 + int(tl[3]-1)
  if b >= c:
    a = int( b*(b+1)/2 + c )
  else :
    a = int( c*(c+1)/2 + b)
  #print (a)
  ti[a] = tl[4]
  tei[a] = tl[4]

#print(ti)
#print(tei)
#print(ti[54])


#Step 4
Done

In [ ]:
# Building the Orthogonalization Matrix

#fs = open("/content/drive/MyDrive/Hartree Fock/s.dat","r")

# Building Overlap Matrix S by reading the file
s = np.zeros((7,7))
for a in fs:
  sl=a.split()
  s[int(sl[0])-1][int(sl[1])-1] = float(sl[2])
  if int(sl[0])!=int(sl[1]):
    s[int(sl[1])-1][int(sl[0])-1] = float(sl[2])
  #print(float(sl[2]))
#print(s)

seval,seigv = np.linalg.eigh(s)   # Finding the eigenvalues and eigenvectors of Overlap Matrix S

#print(seval)
#print(seigv)

s_diag_irt = np.zeros((7,7))
for i in range(7):
  s_diag_irt[i][i] = 1/(math.sqrt(seval[i]))  # Building s^(-1/2); s being the diagonalized matrix of S

s_inv_half = np.zeros((7,7))
a = np.matmul(s_diag_irt,seigv.transpose())
s_inv_half = np.matmul(seigv,a)               # Building the Orthogonalization Matrix S^(-1/2)

#print(s_inv_half)
"""
for i in range(7):
  for j in range(7):
    print(s_inv_half[i][j],end = " ")
  print()
"""











'\nfor i in range(7):\n  for j in range(7):\n    print(s_inv_half[i][j],end = " ")\n  print()\n'

#Step 5
Done

In [ ]:
# Building the initial guess density matrix

fprime = np.zeros((7,7))
a = np.matmul(hc,s_inv_half)
fprime = np.matmul(s_inv_half.transpose(),a)   # Building the initial (guess) fock matrix
print('Fock Matrix:')
print(fprime)

orb_eng,cprime= np.linalg.eigh(fprime)   # cprime (the coefficients) are the coefficients of the eigenvectors (C0 prime).
c0 = np.zeros((7,7))
c0 = np.matmul(s_inv_half,cprime)        # Transforming the eigenvectors into the original (non-orthogonal) AO basis

#print('cprime:')
#print(cprime)
#print('c0:')
#print(c0)

d = np.zeros((7,7))
for i in range(7):
  for j in range(7):
    for k in range(5):             # k loops till range(5),because we're summing only over the occupied MOs.
      d[i][j]+=c0[i][k]*c0[j][k]   # Building the density matrix using the occupied MOs

#print('Density Matrix:')
#print(d)




Fock Matrix:
[[-3.22545866e+01 -2.79149094e+00  6.04031500e-16  8.61098119e-03
   1.90278865e-17 -1.81296650e-01 -1.81296650e-01]
 [-2.79149094e+00 -8.23688912e+00  2.90354845e-15 -2.28292573e-01
  -2.22332556e-16 -3.85798683e-01 -3.85798683e-01]
 [ 5.98320015e-16  2.96162389e-15 -7.54288904e+00 -3.31601039e-16
  -3.79137711e-16 -1.13212060e-01  1.13212060e-01]
 [ 8.61098119e-03 -2.28292573e-01 -3.35721763e-16 -7.45702949e+00
  -1.18462559e-16 -1.10219600e-01 -1.10219600e-01]
 [ 1.90278865e-17 -2.22332556e-16 -3.79137711e-16 -1.18462559e-16
  -7.34714487e+00  3.30556198e-16  7.36569710e-16]
 [-1.81296650e-01 -3.85798683e-01 -1.13212060e-01 -1.10219600e-01
   3.30556198e-16 -4.03295470e+00 -4.46465985e-02]
 [-1.81296650e-01 -3.85798683e-01  1.13212060e-01 -1.10219600e-01
   7.36569710e-16 -4.46465985e-02 -4.03295470e+00]]


#Step 6
Done.

In [ ]:
# Computing the initial SCF Energy

e_elec0 = 0.0

for i in range(7):
  for j in range(7):
    e_elec0 += d[i][j]*(hc[i][j]) + d[i][j]*(hc[i][j])  # SCF electronic energy computed using the density matrix
#print('e_elec0:',e_elec0)
#fenuc = open("/content/drive/MyDrive/Hartree Fock/enuc.dat","r")
enuc = float(fenuc.read())
#print('enuc:',enuc)
e_tot0 = e_elec0 + enuc       # Total energy = Sum of the electronic energy and the nuclear repulsion energy
#print(e_tot0)


#Step 7
Done. But try accessing the two electron integrals with dictionary (key error was being shown)

In [ ]:
# Computing the New Fock Matrix

def index(a,b):           # Function to compute the compound indices
  if a >= b:
    return a*(a+1)/2 + b
  else :
    return b*(b+1)/2 + a

def new_f(d,tei):
    fnew = np.zeros((7,7))
    for i in range(7):
      for j in range(7):
        fnew[i][j] = hc[i][j]
        for k in range(7):
          for l in range(7):
            ij = index(i,j)
            ik = index(i,k)
            kl = index(k,l)
            jl = index(j,l)
            ijkl = int(index(ij,kl))
            ikjl = int(index(ik,jl))
            fnew[i][j] += d[k][l]*(2.0*tei[ijkl] - tei[ikjl])  # Building the new Fock matrix using previous iteration density

    #print('New Fock Matrix:')
    #print(fnew)
    return fnew
    #new_d(fnew)

#fnew = new_f(d,tei)

#Step 8
Done ( via step 10 )

In [ ]:
# Building the new density matrix
def new_d(fnew):
    a = np.matmul(fnew,s_inv_half)
    fi = np.matmul(s_inv_half.transpose(),a)
    #print('Fock Matrix (i):')
    #print(fi)

    orb_eng  , cprime= np.linalg.eigh(fi)   # cprime (the coefficients) is the coefficient of the eigenvector (C0 prime).
    c0 = np.zeros((7,7))
    c0 = np.matmul(s_inv_half,cprime)
    """
    print('cprime:')
    print(cprime)
    print('c0:')
    print(c0)
    """
    di = np.zeros((7,7))
    for i in range(7):
      for j in range(7):
        for k in range(5):
          di[i][j]+=c0[i][k]*c0[j][k]     # Building current iteration density matrix

    #print('Density Matrix:')
    #print(di)
    return di

#di = new_d(fnew)





#Step 9
Done ( via step 10 )


In [ ]:
# Computing the new SCF Energy

def new_energy(d,hc,fnew):
    e_eleci = 0

    for i in range(7):
      for j in range(7):
        e_eleci += d[i][j]*(hc[i][j] + fnew[i][j])  # Electronic energy of the ith iteration

    fenuc = open("/content/drive/MyDrive/Hartree Fock/enuc.dat","r")
    enuc = float(fenuc.read())

    e_toti = e_eleci + enuc    # Total energy of the ith iteration
    return e_toti , e_eleci
#e_toti = new_energy(di,hc,fnew)

#Step 10
Done.


In [ ]:
# Testing for Convergence

e_toti_prev = e_tot0

delta1 = 1e-12
delta2 = 1e-11
ctr = 0
print("%s    %s            %s          %s        %s" %('Iter','E(elec)','E(tot)','Delta(E)','RMS(D)'))
#print('Iter     E(elec)           E(tot)           Delta(E)       RMS(D)')
print("%2d %15.12f %15.12f" %(ctr,e_elec0,e_tot0))
while True :
  fnew = new_f(d,tei)
  di = new_d(fnew)
  e_toti , e_eleci = new_energy(di,hc,fnew)

  delE = e_toti - e_toti_prev
  rmsd = 0
  for i in range(7):
    for j in range(7):
      rmsd += (di[i][j] - d[i][j])*(di[i][j] - d[i][j])

  rmsd = math.sqrt(rmsd)
  ctr+=1
  print("%2d %15.12f  %15.12f  %15.12f %13.12f" %(ctr,e_eleci,e_toti,delE,rmsd))
  if abs(delE) < delta1 and rmsd < delta2 :   # Checking the energy difference and root-mean-squared-difference for convergence
      break
  e_toti_prev = e_toti
  d = di



Iter    E(elec)            E(tot)          Delta(E)        RMS(D)
 0 -125.842077437699 -117.839710375888
 1 -78.286583284740  -70.284216222929  47.555494152959 1.826673084479
 2 -84.048316253435  -76.045949191625  -5.761732968696 0.479570364860
 3 -82.716965960855  -74.714598899044   1.331350292581 0.086831688906
 4 -82.987140757002  -74.984773695191  -0.270174796147 0.031026136359
 5 -82.938133187513  -74.935766125703   0.049007569488 0.010799283179
 6 -82.946271078699  -74.943904016889  -0.008137891186 0.005254826831
 7 -82.944486784914  -74.942119723104   0.001784293785 0.002438579642
 8 -82.944617252243  -74.942250190433  -0.000130467329 0.001177279531
 9 -82.944503500703  -74.942136438892   0.000113751541 0.000564543180
10 -82.944478930626  -74.942111868815   0.000024570077 0.000272204992
11 -82.944461627685  -74.942094565874   0.000017302941 0.000131180309
12 -82.944454198375  -74.942087136564   0.000007429310 0.000063275666
13 -82.944450445051  -74.942083383241   0.000003753324 